> **Tip**: Welcome to the Investigate a Dataset project! You will find tips in quoted sections like this to help organize your approach to your investigation. Once you complete this project, remove these **Tip** sections from your report before submission. First things first, you might want to double-click this Markdown cell and change the title so that it reflects your dataset and investigation.

# Project: Investigate the TMDb Movie Data

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction
This analysis looks into the relations that genre, release year, and budget (adjusted for inflation) have with a movies' overall rating and profit based on the data from The Movie Database (TMDb), which includes information, classifications, and statistics about nearly 11,000 movies.
### Dataset Description 


### Question(s) for Analysis
The questions to be answered are:

1-What is the most highly rated movie genre in each year?
2-Which movie genre is the most profitable on average by year?
3-How much did the average budget of the genres' movies appear to influence the ratings of the movies?
Please note that the explanations for the executed code will precede the code itself throughout the report.

Begin by importing the libraries for needed for analysis and set inline plotting.



In [ ]:
# Use this cell to set up import statements for all of the packages that you
#   plan to use.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

%matplotlib inline
# Remember to include a 'magic word' so that your visualizations are plotted
#   inline with the notebook. See this page for more:
#   http://ipython.readthedocs.io/en/stable/interactive/magics.html


In [ ]:
# Upgrade pandas to use dataframe.explode() function. 
!pip install --upgrade pandas==0.25.0

<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you **document your data cleaning steps in mark-down cells precisely and justify your cleaning decisions.**


### General Properties
Read the .csv file into a pandas dataframe.



In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.
df = pd.read_csv('tmdb-movies.csv')
df.head()



### Data Cleaning
Drop the columns that are not useful for this analysis. Budget may help account for higher ratings, and revenue allows us to calculate profit. The release_year, genres, budget_adj, and vote_average columns are the main columns of information we need to answer the questions posed.

 

In [ ]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.
df.drop(['id', 'imdb_id', 'popularity', 'original_title', 'cast', 'homepage', 'director', 'tagline', 'keywords', 'overview', 'runtime', 'production_companies', 'release_date', 'vote_count'], axis = 1, inplace = True)
df.info()
df.dropna(axis = 0, inplace = True, subset = ['genres'])
df.info()



<a id='eda'></a>
## Exploratory Data Analysis




What is the most popular movie genre for each year?
The genre column in this dataframe is made up of a string of genre names separated by pipes, or the | character.

To explore the genres, we need to divide the movies into groups based on genres. Since each movie can have multiple genres, the simplest way to analyze genre information is to include a movie in the group for each genre it has, even if that means that a movie is included in multiple dataframes.

This does limit the report because it will not look at every combination of genre as separate groups. .nunique() shows us that there are 2,039 different combinations of genres in this dataset; far too many to analyze each combination separately in this report.



In [1]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.
df.genres.nunique()
genre_array = df.genres.values
genre_array = genre_array.astype('U')
print(genre_array.dtype)
split_genre_array = np.core.defchararray.split(a = genre_array, sep = '|')
print(split_genre_array.dtype)
print(split_genre_array)
total_words = 0
max_length = 0

for index, row in enumerate(split_genre_array):
    for word in split_genre_array[index]:
        total_words += 1
        if len(word) > max_length:
            max_length = len(word)
        
print('total_words: ' + str(total_words))
print('max_length: ' + str(max_length))
combined_genre_array = np.empty((1, total_words), dtype = ('U' + str(max_length)))
combined_genre_array.shape
count = 0

for index, row in enumerate(split_genre_array):
    row_list = ast.literal_eval(str(row))
    
    for word in row_list:
        combined_genre_array[0, count] = word
        count += 1
        
print(combined_genre_array)
genre_list = np.unique(combined_genre_array)
print(genre_list)
genre_count = []
for genre in genre_list:
    temp = genre.lower() + "_df"
    temp = temp.replace(" ", "_")
    number = len(eval(temp + '.index'))
    
    genre_count.append([temp[:-3], number])
    
genre_count.sort(key = lambda x:x[1], reverse = True)

for index, genre in enumerate(genre_count):
    genre_count[index] = [genre[0].title().replace("_", " "), genre[1]]

print(genre_count)
plt.figure(figsize=(20, 10))
x, y = [*zip(*genre_count)]
graph = plt.bar(x, y)
plt.xticks(rotation = 'vertical')
plt.xlabel('Genres')
plt.ylabel('Number of Movies')


# Place the values above the bars.
for p in graph.patches:
    plt.annotate(p.get_height(), (p.get_x() + p.get_width() / 2, p.get_height()), ha = 'center', va = 'center', fontsize = 11, xytext = (0, 10), textcoords = 'offset points')

plt.show()
genre, amount = [*zip(*genre_count)]
plt.figure(figsize = (10, 10))
plt.title('Proportion of the Total Number of Movies in Each Genre')
plt.pie(amount, labels = genre, textprops = {'fontsize': 10})

plt.show()
years = df['release_year'].unique()
years.sort()
print(years)
result_rows = ['vote_average_mean', 'budget_adj_mean', 'revenue_adj_mean', 'profit_adj_mean']

for year in years:
    result_rows.append(str(year) + '_vote_average')
    result_rows.append(str(year) + '_budget_adj')
    result_rows.append(str(year) + '_revenue_adj')
    result_rows.append(str(year) + '_profit_adj')
    
    
results_df = pd.DataFrame(index = result_rows, columns = genre_list)
for genre in genre_list:
    # Create the string to eval the dataframe of the current genre's movies.
    genre_df = genre.lower().replace(' ', '_') + '_df'
    
    # Set the overall means for the vote_average, budget_adj, and the revenure_adj columns, and calculate the overall profit mean.
    results_df.at['vote_average_mean', str(genre)] = eval(genre_df)['vote_average'].mean()
    results_df.at['budget_adj_mean', str(genre)] = eval(genre_df)['budget_adj'].mean()
    results_df.at['revenue_adj_mean', str(genre)] = eval(genre_df)['revenue_adj'].mean()
    results_df.at['profit_adj_mean', str(genre)] = (eval(genre_df)['revenue_adj'].mean() - eval(genre_df)['budget_adj'].mean())
    
    # Set these four values for each year.
    for year in years:
        temp_year_df = eval(genre_df).loc[eval(genre_df)['release_year'] == year]
        results_df.at[str(year) + '_vote_average', str(genre)] = temp_year_df['vote_average'].mean()
        results_df.at[str(year) + '_budget_adj', str(genre)] = temp_year_df['budget_adj'].mean()
        results_df.at[str(year) + '_revenue_adj', str(genre)] = temp_year_df['revenue_adj'].mean()
        results_df.at[str(year) + '_profit_adj', str(genre)] = (temp_year_df['revenue_adj'].mean() - temp_year_df['budget_adj'].mean())
    
results_df.info()
results_df = results_df.apply(pd.to_numeric, axis = 1, errors = 'coerce')
results_df.info()
results_df
results_max = results_df.idxmax(axis = 1)
vote_average_max = results_max[0::4]
budget_average_max = results_max[1::4]
revenue_average_max = results_max[2::4]
profit_average_max = results_max[3::4]
print(vote_average_max)
# Use [1:] to avoid counting the overall mean in the graphs.
vote_x = vote_average_max[1:].value_counts().index.values.tolist()
vote_y = vote_average_max[1:].value_counts().tolist()
graph = plt.bar(vote_x, vote_y)

plt.title('Number of Years Genres Have the Highest Vote Average')
plt.xticks(rotation = 'vertical')
plt.yticks([0, 5, 10, 15, 20, 25, 30, 35])
plt.xlabel('Movie Genres')
plt.ylabel('Number of Years')

# Place the values above the bars.
for p in graph.patches:
    plt.annotate(p.get_height(), (p.get_x() + p.get_width() / 2, p.get_height()), ha = 'center', va = 'center', fontsize = 11, xytext = (0, 10), textcoords = 'offset points')

plt.show()
plt.figure(figsize = (10, 10))
plt.title('Proportion of Years Genres Have the Highest Vote Average')
plt.pie(vote_y, labels = vote_x, textprops = {'fontsize': 10})

plt.show()
print(budget_average_max)
# Use [1:] to avoid counting the overall mean in the graphs.
budget_x = budget_average_max[1:].value_counts().index.values.tolist()
budget_y = budget_average_max[1:].value_counts().tolist()
graph = plt.bar(budget_x, budget_y)

plt.title('Number of Years Genres Have the Highest Average Budget')
plt.xticks(rotation = 'vertical')
plt.yticks([0, 5, 10, 15])
plt.xlabel('Movie Genres')
plt.ylabel('Number of Years')

# Place the values above the bars.
for p in graph.patches:
    plt.annotate(p.get_height(), (p.get_x() + p.get_width() / 2, p.get_height()), ha = 'center', va = 'center', fontsize = 11, xytext = (0, 6), textcoords = 'offset points')

plt.show()
plt.figure(figsize = (10, 10))
plt.title('Proportion of Years Genres Have the Highest Average Budget')
plt.pie(budget_y, labels = budget_x, textprops = {'fontsize': 10})

plt.show()
print(revenue_average_max)
Use [1:] to avoid counting the overall mean in the graphs.
revenue_x = revenue_average_max[1:].value_counts().index.values.tolist()
revenue_y = revenue_average_max[1:].value_counts().tolist()
graph = plt.bar(revenue_x, revenue_y)

plt.title('Number of Years Genres Have the Highest Average Revenue')
plt.xticks(rotation = 'vertical')
plt.yticks([0, 5, 10, 15, 20])
plt.xlabel('Movie Genres')
plt.ylabel('Number of Years')

# Place the values above the bars.
for p in graph.patches:
    plt.annotate(p.get_height(), (p.get_x() + p.get_width() / 2, p.get_height()), ha = 'center', va = 'center', fontsize = 11, xytext = (0, 10), textcoords = 'offset points')

plt.show()
plt.figure(figsize = (10, 10))
plt.title('Proportion of Years Genres Have the Highest Average Revenue')
plt.pie(revenue_y, labels = revenue_x, textprops = {'fontsize': 10})

plt.show()

SyntaxError: invalid syntax (<ipython-input-1-d97f083b6531>, line 153)

What is the most profitable movie genre for each year?¶


In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.
print(profit_average_max)
# Use [1:] to avoid counting the overall mean in the graphs.
profit_x = profit_average_max[1:].value_counts().index.values.tolist()
profit_y = profit_average_max[1:].value_counts().tolist()
graph = plt.bar(profit_x, profit_y)

plt.title('Number of Years Genres Have the Highest Average Profit')
plt.xticks(rotation = 'vertical')
plt.yticks([0, 5, 10, 15, 20])
plt.xlabel('Movie Genres')
plt.ylabel('Number of Years')

# Place the values above the bars.
for p in graph.patches:
    plt.annotate(p.get_height(), (p.get_x() + p.get_width() / 2, p.get_height()), ha = 'center', va = 'center', fontsize = 11, xytext = (0, 10), textcoords = 'offset points')

plt.show()
plt.figure(figsize = (10, 10))
plt.title('Proportion of Years Genres Have the Highest Average Profit')
plt.pie(profit_y, labels = profit_x, textprops = {'fontsize': 10})

plt.show()


<a id='conclusions'></a>
## Conclusions

This report has analyzed the The Movie Database (TMDb) to determine the answers to these three questions.

What is the most highly rated movie genre in each year?
Which movie genre is the most profitable on average by year?
How much did the average budget of the genres' movies appear to influence the ratings of the movies?
The most highly rated movie genre by year varied, but the Documentary genre was the genre that had the highest average rating across the most years. This may be because documentaries are more serious productions that tend to be polished and because less of them are produced.

The most profitible movie genre by year varied as well, but the Adventure genre was the genre that had the highest average profit across the most years. This may be because this genre also tended to have the highest budget, but also because it is one of the more popular movie genres that have a wider audience.

Finally, a higher average budget does appear to have a very slight association with a higher average vote rating, but more statistical analysis will need to be performed to prove anything. Additionally, the highest and lowest vote ratings of all the dataset were with lower budget movies.

With this information it can be determined that in order to make a more successful movie measured by voted ratings, one should make a documentary with the highest budget possible, and that to make a more successful movie by profit or revenue, one should make adventure movies with higher budgets when able.

Further analysis would be beneficial by factoring the vote_count values into the calculations to eliminate bias or statistical outliers in movies with very high or very low vote_average values but low vote_count values. Additionally, this analysis is limited because of how the averages of each year and overall were divided and compared by individual genre. Doing the same for genre pairs would give more insightful information into the nuances of genre popularity and success.





In [2]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])

0